<a href="https://colab.research.google.com/github/Hanna413/Parsing_GB/blob/GoodCode_Examples/%D0%9F%D1%80%D0%B8%D0%BC%D0%B5%D1%80_%D0%BF%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3%D0%B0_%D1%81_hh_Erik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep

In [3]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

URL_FIRST_PAGE_HH = 'https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20'

In [4]:
headers = {
    'User-Agent': USER_AGENT,
}

In [5]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
      print('Парсинг завершен')
      return result
    else:
      print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://spb.hh.ru' + link_next_page['href']
    else:
      print('Парсинг завершен')
      return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy_hh(dom_vacancy):
  sleep(0.1)
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'hh.ru',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [6]:
result = parse_hh(URL_FIRST_PAGE_HH, headers)

Cтраница №1, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20
Cтраница №2, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20&page=1&hhtmFrom=vacancy_search_list
Cтраница №3, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20&page=2&hhtmFrom=vacancy_search_list
Cтраница №4, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20&page=3&hhtmFrom=vacancy_search_list
Cтраница №5, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=descripti

In [7]:
len(result)


166

In [8]:
result


[{'vacancy_name': 'Middle Python developer (Казань)',
  'vacancy_salary': 'от 160\u202f000 руб.',
  'min_salary': 160000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://spb.hh.ru/vacancy/76115939?from=vacancy_search_list&query=python',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Lead Python developer',
  'vacancy_salary': 'до 500\u202f000 руб.',
  'min_salary': None,
  'max_salary': 500000,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://spb.hh.ru/vacancy/77337810?from=vacancy_search_list&query=python',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Junior/Middle Backend-разработчик на Ruby',
  'vacancy_salary': '110\u202f000 – 150\u202f000 руб.',
  'min_salary': 110000,
  'max_salary': 150000,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://spb.hh.ru/vacancy/76964796?from=vacancy_search_list&query=python',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Разработчик Python',
  'vacancy_salary': 'от 120\u202f000 руб.',
  'min_salary